河野君の見付けた d-surface のモジュールの可視化をこころみる

Clustice を使えば、グラフを氷の構造にできる、はず。


In [ ]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx
import py3Dmol

import genice_core
from clustice.serialize import serialize
from clustice.geometry import make_layout
from clustice import gromacs, yaplot
from clustice.water import tip4p, spce
from clustice import graph
from cycless import cycles

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

g = nx.Graph(
    [
        [0, 20],
        [0, 23],
        [1, 2],
        # [2, 3],
        # [3, 4],
        [3, 22],
        [3, 18],
        [4, 1],
        [5, 6],
        # [6, 7],
        # [7, 8],
        [19, 6],
        [8, 21],
        [8, 5],
        [9, 10],
        [10, 11],
        [11, 12],
        [12, 9],
        [13, 14],
        [14, 15],
        [15, 16],
        [16, 13],
        [1, 17],
        [17, 5],
        [17, 20],
        [17, 23],
        [20, 19],
        # [19, 8],
        [7, 19],
        [19, 9],
        [20, 18],
        [18, 4],
        [18, 10],
        # [6, 21],
        [7, 21],
        [21, 13],
        [21, 23],
        # [23, 7],
        [23, 22],
        [22, 2],
        [22, 14],
        [12, 24],
        [24, 16],
        [11, 25],
        [25, 15],
        [25, 0],
        [24, 0],
        [3, 25],
        [7, 24],
        [6, 26],
        [7, 26],
        [8, 26],
        [2, 27],
        [3, 27],
        [4, 27],
        [9, 28],
        [10, 29],
        [11, 30],
        [12, 31],
        [28, 29],
        [29, 30],
        [30, 31],
        [31, 28],
        [13, 32],
        [14, 33],
        [15, 34],
        [16, 35],
        [32, 33],
        [33, 34],
        [34, 35],
        [35, 32],
    ]
)
# count cycles in the graph
cyclehist = {}
for cycle in cycles.cycles_iter(g, maxsize=5):
    nnode = len(cycle)
    cyclehist[nnode] = cyclehist.get(nnode, 0) + 1
print(cyclehist)


if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    mol_positions = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    mol_positions = make_layout(g, edge_length=L)


yap = yaplot.render(g, mol_positions)
with open("dmodule.yap", "w") as f:
    f.write(yap)


# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = genice_core.ice_graph(
    g, vertexPositions=mol_positions, dipoleOptimizationCycles=1000
)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = gromacs.render(
    dg,
    mol_positions,
    water_model=spce,
    cell_matrix=np.array([[30.0, 0.0, 0.0], [0.0, 30.0, 0.0], [0.0, 0.0, 30.0]]),
    shift=np.array([0.5, 0.5, 0.5]),
)
with open("dmodule.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})  #: {"radius": 0.75}})
view.addUnitCell()
view.zoomTo()
view.show()
view.png()

{4: 9, 5: 16}


3Dmol.js failed to load for some reason. Please check your browser console for error messages.